<a href="https://colab.research.google.com/github/RockhoRockho/Data-project/blob/main/18_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN
* 입력과 출력을 시퀀스 단위로 처리하는 스퀀스 모델
* 순환 신경망(Recurrent Neural Network)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from glob import glob
from tqdm import tqdm

In [ ]:
import cv2
import os
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
model = keras.models.load_model(
    'drive/MyDrive/ucf11_weights/10-0.98.hdf5',
    custom_objects={'KerasLayer': hub.KerasLayer}
)

model.build([None, 256, 256, 3])
model.summary()

In [ ]:
feature_extractor = keras.Sequential(
    [
     keras.Input(shape=(256, 256, 3)),
     model.layers[0]
    ], name='feature_extractor'
)

feature_extractor.summary()

In [ ]:
os.mkdir('drive/MyDrive/UCF11_updated_npy')
os.mkdir('drive/MyDrive/UCF11_updated_npy/train')
os.mkdir('drive/MyDrive/UCF11_updated_npy/valid')

In [ ]:
max_frames = 10
SAVE_DIR = 'drive/MyDrive/UCF11_updated_npy/'
train_df = pd.read_csv('drive/MyDrive/ucf11_train_vid.csv')
valid_df = pd.read_csv('drive/MyDrive/ucf11_valid_vid.csv')

In [ ]:
for i, elem in tqdm(train_df.iterrows(), total=len(train_df)):
  label = elem['label']
  cap = cv2.VideoCapture(
      elem['file_path']
      
  )
  
  frames = []
  while True:
    ret, frame = cap.read()
    if not ret:
      break

    frame_ = frame.copy()
    frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
    frame_ = cv2.resize(frame_, (256, 256))
    frame_ = frame_.astype('float32')
    frame_ = frame_ / 255.

    frames.append(frame_)
    if len(frames) == max_frames:
      break
    
  cap.release()
  
  frames = np.array(frames)
  features = feature_extractor.predict(frames)

  file_name = SAVE_DIR + f'train/{label}_{i}.npy'
  np.save(file_name, features)

In [ ]:
print(len(glob(SAVE_DIR + 'train/*')))

In [ ]:
for i, elem in tqdm(valid_df.iterrows(), total=len(valid_df)):
  label = elem['label']
  cap = cv2.VideoCapture(
      elem['file_path']
      
  )
  
  frames = []
  while True:
    ret, frame = cap.read()
    if not ret:
      break

    frame_ = frame.copy()
    frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
    frame_ = cv2.resize(frame_, (256, 256))
    frame_ = frame_.astype('float32')
    frame_ = frame_ / 255.

    frames.append(frame_)
    if len(frames) == max_frames:
      break
    
  cap.release()
  
  frames = np.array(frames)
  features = feature_extractor.predict(frames)

  file_name = SAVE_DIR + f'valid/{label}_{i}.npy'
  np.save(file_name, features)

In [ ]:
print(len(glob(SAVE_DIR + 'valid/*')))